# AI HUB 데이터셋 정제

### 저음질 전화망 음성인식 데이터
- URL : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=571

- 사용한 데이터 셋 : 
    1. `dataset/1.Training/라벨링데이터_230316/D02/J05`
        - Training 데이터에서 창원시, 전주시 민원 데이터 중 복지 데이터 
    2. `dataset/2.Validation/라벨링데이터_230316/D02/J05`
        - Validation 데이터에서 창원시, 전주시 민원 데이터 중 복지 데이터 

In [1]:
from glob import glob

# 민원 데이터 중 복지 데이터 디렉터리 경로
directory = glob('./J05/*/*.json')

In [2]:
import json

li = []

# 파일 내용 읽기
for file in directory:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        li.append(data)

In [3]:
# 전체 데이터 경로 
dialog = [i['dataSet']['dialogs'] for i in li]

sentence = []
for i in dialog:
    sentence.append([{j['speaker']:j['text']} for j in i])
print("추출된 데이터 갯수 :", len(sentence))

추출된 데이터 갯수 : 8043


In [4]:
# 대화 합치기 함수
def merge_conversations(conversation):
    merged_conversation = []
    current_speaker = None
    current_text = ""
    
    for line in conversation:
        speaker, text = list(line.items())[0]  # 키와 값을 가져옴
        
        # 같은 사람이 말할 경우 대화를 이어붙임
        if speaker == current_speaker:
            current_text += " " + text
        else:
            # 다른 사람일 경우 현재 대화를 리스트에 추가하고 새로 시작
            if current_speaker is not None:
                merged_conversation.append({current_speaker: current_text})
            current_speaker = speaker
            current_text = text
    
    # 마지막 대화 추가
    if current_speaker is not None:
        merged_conversation.append({current_speaker: current_text})
    
    return merged_conversation

# 대화 합치기 실행
merged_conversation = [merge_conversations(i) for i in sentence]
print("현재 저장된 데이터 :", len(merged_conversation))

현재 저장된 데이터 : 8043


In [5]:
# json 파일 이름
JSON_NAME = "civil.json"

# JSON 파일로 저장
with open(JSON_NAME, 'w', encoding='utf-8') as f:
    json.dump(merged_conversation, f, 
              ensure_ascii=False, 
              indent=4 # 들여쓰기
              )

In [11]:
# 원천 데이터 / 합친 데이터 
len([j for i in sentence for j in i]), len([j for i in merged_conversation for j in i])

(263593, 149682)